## 1-D CNN for Chagas ECG detection

This notebook adds a straightforward convolutional-neural-network baseline.  
It treats each ECG as a 12-channel time-series, using stacked Conv1D blocks
followed by global pooling and a dense sigmoid output.

Goals:  
- Capture local waveform patterns that tree models miss  
- Provide a TensorFlow reference architecture before experimenting with more
  complex ConvNeXt-style models  
- Keep preprocessing, splits, and metrics identical to prior notebooks for an
  apples-to-apples comparison

## Environment setup

### Import libraries

In [1]:
import random
import numpy as np

import tensorflow as tf
from keras import layers, models    

from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    average_precision_score,
    precision_recall_fscore_support,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
import matplotlib.pyplot as plt

RANDOM_STATE = 2025
tf.keras.utils.set_random_seed(RANDOM_STATE)

### Load preprocessed datasets

In [2]:
# Path to the folder containing preprocessed data
DATA_DIR = 'data/prepared'

# train = np.load(f'{DATA_DIR}/train_full_parts0-6.npz')
train = np.load(f'{DATA_DIR}/train_bal_parts0-6_aug.npz')
val = np.load(f'{DATA_DIR}/val_parts0-6.npz')
test = np.load(f'{DATA_DIR}/test_external.npz')

# Extract arrays and labels from the loaded data
X_train, y_train = train['X'], train['y']
X_val, y_val = val['X'], val['y']
X_test, y_test = test['X'], test['y']

# Check array shapes and positive counts
print('Train :', X_train.shape,  'Positives:', y_train.sum())
print('Val   :', X_val.shape,    'Positives:', y_val.sum())
print('Test  :', X_test.shape,   'Positives:', y_test.sum())

Train : (17880, 2920, 12) Positives: 4470
Val   : (27873, 2920, 12) Positives: 559
Test  : (23430, 2920, 12) Positives: 1631


## Modeling

### Build 1-D CNN model


In [3]:
def build_cnn_model(seq_len=2920, n_ch=12):
    """Stacked Conv-BN-ReLU blocks in Sequential style."""
    tf.keras.backend.clear_session()
    tf.random.set_seed(RANDOM_STATE)

    # Create a Sequential model
    model = tf.keras.Sequential(name="ecg_cnn_seq")

    # Input layer
    model.add(layers.Input(shape=(seq_len, n_ch)))

    # 1st conv block
    model.add(layers.Conv1D(32, 7, padding='same',
                            activation='relu',
                            input_shape=(seq_len, n_ch)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Dropout(0.3))

    # 2nd conv block
    model.add(layers.Conv1D(64, 5, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Dropout(0.3))

    # 3rd conv block
    model.add(layers.Conv1D(128, 5, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Dropout(0.3))

    # 4th conv block
    model.add(layers.Conv1D(256, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Dropout(0.3))

    # global pooling → sigmoid
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auroc', curve='ROC'),
                 tf.keras.metrics.AUC(name='auprc', curve='PR')]
    )

    model.summary(line_length=80)
    return model


model = build_cnn_model()

c:\Users\jak50\anaconda3\envs\ds207-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "ecg_cnn_seq"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                      ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                   │ (None, 2920, 32)         │         2,720 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization               │ (None, 2920, 32)         │           128 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)      │ (None, 1460, 32)         │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout (Dropout)                 │ (None, 1460, 32)         │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)                 │ (None, 1460, 64)         │        10,304 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_1             │ (None, 1460, 64)         │           256 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)    │ (None, 730, 64)          │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout_1 (Dropout)               │ (None, 730, 64)          │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)                 │ (None, 730, 128)         │        41,088 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_2             │ (None, 730, 128)         │           512 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)    │ (None, 365, 128)         │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout_2 (Dropout)               │ (None, 365, 128)         │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)                 │ (None, 365, 256)         │        98,560 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ batch_normalization_3             │ (None, 365, 256)         │         1,024 │
│ (BatchNormalization)              │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)    │ (None, 182, 256)         │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout_3 (Dropout)               │ (None, 182, 256)         │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ global_average_pooling1d          │ (None, 256)              │             0 │
│ (GlobalAveragePooling1D)          │                          │               │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense (Dense)                     │ (None, 1)                │           257 │
└───────────────────────────────────┴──────────────────────────┴───────────────┘

 Total params: 154,849 (604.88 KB)

 Trainable params: 153,889 (601.13 KB)

 Non-trainable params: 960 (3.75 KB)

### Train the CNN


In [ ]:
# Convert data to float32 for training
X_train_tf = X_train.astype('float32')
X_val_tf   = X_val.astype('float32')

# Early stopping callback 
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_auroc',
    mode='max',
    patience=5,
    restore_best_weights=True,
)

# Set training parameters
EPOCHS = 30
BATCH = 64

# Train the model
history = model.fit(
    X_train_tf, y_train,
    validation_data=(X_val_tf, y_val),
    epochs=EPOCHS,
    batch_size=BATCH,
    callbacks=[early_stop],
    verbose=2,
)

Epoch 1/30
280/280 - 332s - 1s/step - auprc: 0.5294 - auroc: 0.7476 - loss: 0.4818 - val_auprc: 0.0889 - val_auroc: 0.7859 - val_loss: 0.2638
Epoch 2/30
280/280 - 245s - 877ms/step - auprc: 0.5930 - auroc: 0.7975 - loss: 0.4461 - val_auprc: 0.0952 - val_auroc: 0.7983 - val_loss: 0.2765
Epoch 3/30
280/280 - 275s - 982ms/step - auprc: 0.6108 - auroc: 0.8089 - loss: 0.4364 - val_auprc: 0.0988 - val_auroc: 0.8041 - val_loss: 0.2947
Epoch 4/30
280/280 - 223s - 797ms/step - auprc: 0.6199 - auroc: 0.8155 - loss: 0.4306 - val_auprc: 0.0976 - val_auroc: 0.8062 - val_loss: 0.2802
Epoch 5/30
280/280 - 179s - 640ms/step - auprc: 0.6302 - auroc: 0.8215 - loss: 0.4250 - val_auprc: 0.0966 - val_auroc: 0.8066 - val_loss: 0.2806
Epoch 6/30
280/280 - 205s - 733ms/step - auprc: 0.6382 - auroc: 0.8237 - loss: 0.4219 - val_auprc: 0.0994 - val_auroc: 0.8100 - val_loss: 0.2833
Epoch 7/30
280/280 - 229s - 819ms/step - auprc: 0.6418 - auroc: 0.8275 - loss: 0.4188 - val_auprc: 0.0980 - val_auroc: 0.8085 - val_l

### Evaluation

In [ ]:
def keras_report(model, name, X_split, y_split, plot_cm=True):
    """
    Compute metrics for a Keras binary-classifier (sigmoid output).
    Args:
        name (str): Name of the dataset split (e.g., 'Train', 'Validation', 'External test').
        X_split (np.ndarray): Feature matrix for the split.
        y_split (np.ndarray): True labels for the split.
        plot_cm (bool): Whether to plot the confusion matrix.
    Returns:
        Prints the performance metrics and confusion matrix.
    """
    y_prob = model.predict(X_split, verbose=0).squeeze()
    y_pred = (y_prob >= 0.5)

    acc   = accuracy_score(y_split, y_pred)
    auroc = roc_auc_score(y_split, y_prob)
    auprc = average_precision_score(y_split, y_prob)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_split, y_pred, average='binary', zero_division=0
    )
    tn, fp, fn, tp = confusion_matrix(y_split, y_pred).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    print(f'{name} metrics')
    print(f'  accuracy     {acc:.3f}')
    print(f'  AUROC        {auroc:.3f}')
    print(f'  AUPRC        {auprc:.3f}')
    print(f'  precision    {prec:.3f}')
    print(f'  recall       {rec:.3f}')
    print(f'  specificity  {specificity:.3f}')
    print(f'  F1           {f1:.3f}\n')

    if plot_cm:
        ConfusionMatrixDisplay(
            confusion_matrix(y_split, y_pred),
            display_labels=['Neg', 'Pos'],
        ).plot(cmap='Blues')
        plt.title(f'{name} confusion matrix')
        plt.show()

In [ ]:
# Convert data to float32 for evaluation
X_train_tf = X_train.astype('float32')
X_val_tf   = X_val.astype('float32')
X_test_tf  = X_test.astype('float32')

# Evaluate the model on different splits
keras_report(model, 'Train', X_train_tf, y_train)
keras_report(model, 'Validation', X_val_tf, y_val)
keras_report(model, 'External test', X_test_tf, y_test)

In [ ]:
# Plot training curves
plt.figure(figsize=(6,4))
plt.plot(history.history['auroc'], label='train AUROC')
plt.plot(history.history['val_auroc'], label='val AUROC')
plt.xlabel('epoch')
plt.ylabel('AUROC')
plt.legend()
plt.grid(True)
plt.title('Training progress')
plt.show()